In [1]:
from tensorflow.keras import layers, models, Model
import numpy as np
import pandas as pd

ModuleNotFoundError: No module named 'tensorflow.keras'

In [ ]:
# Import the data
path = 'https://static.bc-edx.com/ai/ail-v-1-0/m19/lesson_3/datasets/wine_quality.csv'
df = pd.read_csv(path)
df.head()

We have two choices; should the y columns of quality and color all be predicted in a single layer? It might be easier to separate these into two layers, both having the sigmoid activation function. To start, lets preprocess these layers using labelencoder and onehotencoder.

In [ ]:
# Preprocess y
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Preprocess "quality" column (one-hot encoding)
df.quality = pd.get_dummies(df.quality)

# Preprocess "color" column (label encoding for binary; one-hot encoding for multiple categories)
label_encoder = LabelEncoder()
df.color = label_encoder.fit_transform(df.color)

# Concatenate the encoded columns to the original DataFrame


# Drop the original "quality" and "color" columns



In [ ]:
df.sample(n=10)

In [5]:
# Split data into X and two separate y variables
y_color = df['color']
y_quality = df['quality']

X = df.drop(columns=['quality', 'color'])

# Split data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_quality_train, y_quality_test,  y_color_train, y_color_test = train_test_split(X, y_color, y_quality, random_state=42)


Now that the data is processed, start by creating the shared portion of the model. We start with the input layer, but note that because we are note using the "sequential" model from keras, we must specify where each new layer attaches to the model. We do this by placing the name of the preceding layer in parentheses at the end of the line creating the new layer.

Note how shared_layer1 is created as a Dense layer, and then (input_layer) specifies that shared_layer1 will come directly after the input_layer. In the following line, shared_layer2 is created to follow shared_layer1.

In [6]:
# Create the shared layers of the model

# Input layer
input_layer = layers.Input(shape=(X.shape[1],), name='Input_layer')
# Shared hidden layers
shared_layer1 = layers.Dense(64, activation='relu', name='shared layer 1')(input_layer)
shared_layer2 = layers.Dense(32, activation='relu', name='shared layer 2')(shared_layer1)


So far, we've created a sequential set on layers, one following the other. Now we will create two branches for our two output layers. To do this, we create two layers and specify the same "preceding layer" for each. Note how both the quality_output layer and the color_output layer connect to the model via shared_layer2.

When creating these layers, we determine that sigmoid is best for quality and for color. That said, there are arguments to be made for either!

In [7]:
# Branch for quality prediction
quality_output = layers.Dense(3 ,activation="softmax")(shared_layer2)
# Branch for color prediction
color_output = layers.Dense(1 ,activation="sigmoid")(shared_layer2)

Now we can pull the model together. We only need to specify the input and output layers and Keras will infer the rest. To compile the model, we can specify unique metrics and loss functions for each output layer, but for this dataset we have chosen 'binary_crossentropy' as the loss function and 'accuracy' as the metric for both output layers.

In [ ]:
# Create the model
model = Model(inputs=input_layer, outputs=[quality_output, color_output])

# Compile the model
model.compile(
	optimizer='adam',
	loss='binary_crossentropy',
	metric=['accuracy']
)

# Display the model summary
model.summery()

To fit the model to the data, we specify X_train as normal, but pass a dictionary for the y_data.

In [ ]:
# Fit the model
model.fit(
	x=X_train,
	y=[y_color, y_quality],
	batch_size=None,
	epochs=10,
	verbose=1
)

In [ ]:
# Evaluate the model with the testing data

model.evaluate(
	x=X_train,
	y=[y_color, y_quality],
	batch_size=None,
	return_dict=False
)
